In [9]:
import tensorflow as tf
import numpy as np
import math

 
import os
import random
import sklearn



words = {}

def load_input_sentences(filepath):
    sentence_file = open(filepath,'r')
    sentence = sentence_file.readline()
    sentences = []
    solutions = []
    while sentence != '':
        substrings = sentence.split(' ')
        solution = substrings[-1].strip()
        sentence = ''
        for i in range(len(substrings)-1):
            sentence = sentence + substrings[i].strip()+' '    
        solutions.append(int(solution))
        sentences.append(sentence.strip())
        sentence = sentence_file.readline()
        
    return sentences, solutions



def create_vocabulary():
    num = 0
    vocab_size = 0
    for sentence in sentences:
        substrings = sentence.split(' ')
        for string in substrings:
            if words.get(string) == None:
                #print num
                words[string] = num
                num += 1
                vocab_size = num
    
    return vocab_size

def create_input_LSTM(sentences):
    sentence_embeddings = []
    for i in range(len(sentences)):
        sentence = sentences[i]
        substrings = sentence.split(' ')
        for j in range(len(substrings)):
            #print(vocab_size)
            if j == len(sentence_embeddings):
                sentence_embeddings.append([])
            embedding = np.zeros([vocab_size])
            #print(embedding.shape)
            embedding[words.get(substrings[j])] = 1
            sentence_embeddings[j].append(embedding)
    for i in range(len(sentence_embeddings)):
        sentence_embeddings[i] = np.array(sentence_embeddings[i])
    return np.array(sentence_embeddings)
    
def one_hot(input, dimension):
    one_hot_vectors = []
    map = {}
    last_index = 0
    for index in input:
        one_hot_vector = np.zeros([dimension])
        #print(index)
        if (map.get(index) == None):
            map[index] = last_index
            last_index += 1
        one_hot_vector[map.get(index)] = 1
        one_hot_vectors.append(one_hot_vector)
    return np.array(one_hot_vectors)

def fc(input, num_output, name = 'fc'):
    with tf.variable_scope(name):
        num_input = input.get_shape()[1]
        W = tf.get_variable('w', [num_input, num_output], tf.float32, tf.random_normal_initializer(0.0, 0.02))
        b = tf.get_variable('b', [num_output], initializer = tf.constant_initializer(0.0))
        return tf.matmul(input, W) + b

In [10]:
sentences, solutions = load_input_sentences('sentences_nlp.txt')
vocab_size = create_vocabulary()
print(vocab_size)
sentence_embeddings = create_input_LSTM(sentences)
solutions_one_hot = one_hot(solutions,100)
print (sentence_embeddings[3],sentences[3])
print (sentence_embeddings.shape, sentences[3])
#test_sentences, test_solutions = load_input_sentences('sentences_nlp_test.txt')

18
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]] 6 multiplied by 9
(5,) 6 multiplied by 9


In [19]:
num_epochs = 100
total_series_length = 50000
truncated_backprop_length = 15
state_size = vocab_size
num_classes = 100
echo_step = 3
batch_size = 5
num_batches = total_series_length//batch_size//truncated_backprop_length


batchX_placeholder = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length, vocab_size])
batchY_placeholder = tf.placeholder(tf.int32, [batch_size, truncated_backprop_length, vocab_size])

init_state = tf.placeholder(tf.float32, [batch_size, state_size, vocab_size])

W2 = tf.Variable(np.random.rand(state_size, num_classes),dtype=tf.float32)
b2 = tf.Variable(np.zeros((1,num_classes)), dtype=tf.float32)

# Unpack columns
inputs_series = tf.unstack(batchX_placeholder, axis=1)
labels_series = tf.unstack(batchY_placeholder, axis=1)

# Forward passes
cell = tf.nn.rnn_cell.BasicLSTMCell(state_size)
states_series, current_state = tf.contrib.rnn(cell, inputs_series, init_state)

logits_series = [tf.matmul(state, W2) + b2 for state in states_series] #Broadcasted addition
predictions_series = [tf.nn.softmax(logits) for logits in logits_series]

losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels) for logits, labels in zip(logits_series,labels_series)]
total_loss = tf.reduce_mean(losses)

train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss)

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    loss_list = []

    for epoch_idx in range(num_epochs):
        x = sentence_embeddings
        y = solutions_one_hot
        _current_state = np.zeros((batch_size, state_size, vocab_size))

        print("New data, epoch", epoch_idx)

        for batch_idx in range(num_batches):
            start_idx = batch_idx * truncated_backprop_length
            end_idx = start_idx + truncated_backprop_length

            batchX = x[:,start_idx:end_idx]
            batchY = y[:,start_idx:end_idx]

            _total_loss, _train_step, _current_state, _predictions_series = sess.run(
                [total_loss, train_step, current_state, predictions_series],
                feed_dict={
                    batchX_placeholder:batchX,
                    batchY_placeholder:batchY,
                    init_state:_current_state
                })

            loss_list.append(_total_loss)

            if batch_idx%100 == 0:
                print("Step",batch_idx, "Loss", _total_loss)
                plot(loss_list, _predictions_series, batchX, batchY)


TypeError: 'module' object is not callable